<a href="https://colab.research.google.com/github/JoseAfonso41/ML_HealthcareCostPrediction/blob/main/C%C3%B3pia_de_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
data = pd.read_csv('insurance.csv')
data.tail()

In [ ]:
categories = ["sex", "smoker", "region"]
data = pd.get_dummies(data, columns = categories, drop_first=True)
data.head()

In [ ]:
#Split train-test data as 80-20
training_data = data.sample(frac=0.8,random_state=0)
testing_data = data.drop(training_data.index)

In [ ]:
#Pop the y label for train and test sets
training_labels = training_data.pop('expenses')
testing_labels = testing_data.pop('expenses')

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(training_data.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
sample_batch = training_data[:10]
sample_prediction = model.predict(sample_batch)
sample_prediction

In [ ]:
NUM_EPOCHS = 1000

history = model.fit(
  training_data, training_labels,
  epochs=NUM_EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
mae = history.history["mae"]
loss = history.history["loss"]
epochs = history.epoch

plt.figure()
plt.scatter(epochs, mae, label='Mean Absolute Error')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error (MAE)')
plt.xlim([0, NUM_EPOCHS])
plt.ylim([0, 5000])
plt.legend()
plt.show()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Evaluate the model on the test data
test_loss, test_mae, test_mse = model.evaluate(testing_data, testing_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(test_mae))

if test_mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(testing_data).flatten()

plt.figure()
plt.scatter(testing_labels, test_predictions)
plt.xlabel('Actual Expenses')
plt.ylabel('Predicted Expenses')
plt.plot([0, 50000], [0, 50000], color="red")  # Plot a line for perfect predictions
plt.xlim([0, 50000])
plt.ylim([0, 50000])
plt.show()
